# 04) 한국어 BERT의 마스크드 언어 모델(Masked Language Model) 실습

## 1. 마스크드 언어 모델과 토크나이저

In [1]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [2]:
# 'klue/bert-base'는 대표적인 한국어 BERT 
# 해당모델이 파이토치로 학습된 모델이지만 텐서플로우에서 사용하겠다는 의미
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

## 2. 정수 인코딩

In [3]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [4]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[   2, 4713, 2259, 3944, 6001, 2259,    4,  809,   18,    3]])>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [9]:
inputs['input_ids']

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[   2, 4713, 2259, 3944, 6001, 2259,    4,  809,   18,    3]])>

In [10]:
# 세그먼트 인코딩 결과
inputs['token_type_ids']

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [11]:
# 오텐션 마스크
inputs['attention_mask']

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>

## [MASK] 토큰 예측하기

In [12]:
#FillMaskPipeline은 모델과 토크나이저 지정시 마스크드 언어 모델의 예측결과 정리해서 보여준다.
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [13]:
pip('축구는 정말 재미있는 [MASK]다')

[{'score': 0.7453209757804871,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다'},
 {'score': 0.16496725380420685,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다'},
 {'score': 0.019687669351696968,
  'token': 578,
  'token_str': '게',
  'sequence': '축구는 정말 재미있는 게 다'},
 {'score': 0.01724977232515812,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다'},
 {'score': 0.006887007970362902,
  'token': 4171,
  'token_str': '게임',
  'sequence': '축구는 정말 재미있는 게임 다'}]

In [14]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.8382413983345032,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.02827559784054756,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.017189329490065575,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989685267210007,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382631629705429,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [15]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.0801255851984024,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.06124062091112137,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.01748666539788246,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.01613178476691246,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360736288130283,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]